In [1]:
import numpy as np
import pandas as pd
import random
import keras
import csv
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from bayes_opt import BayesianOptimization
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Conv2D,MaxPooling2D,Dense, Flatten,Dropout
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
path = "X:/XXXX/XXXX" # root directory
groups = ["G2","notG2"] # class name
model_name = "Model1" # name of output model w/o extension

nb_ch = 1 # number of input image channels (1 or 2)
single_ch = 0 # if number of channel is 1, choose channel #0 or 1

nb_classes = len(groups)
if(path[-1]=="/"):
    path=path[:-1]

In [3]:
# load training and test data sets
x_train = np.load(path+"/data/x_train.npy")
y_train = np.load(path+"/data/y_train.npy")

x_test = np.load(path +"/data/X_test.npy")
y_test = np.load(path +"/data/Y_test.npy")

# normalize
x_train = x_train/255
x_test = x_test/255

y_train = np_utils.to_categorical(y_train,nb_classes)
y_test = np_utils.to_categorical(y_test,nb_classes)

# shuffle sequence of data
index=list(range(x_train.shape[0]))
index=random.sample(index,len(index))
x_train = x_train[index]
y_train = y_train[index]

# pick color channels
if nb_ch == 2:
    x_train=x_train[...,:2]
    x_test=x_test[...,:2]
else:
    x_train=x_train[...,single_ch]
    x_test=x_test[...,single_ch]
    x_train=x_train[...,None]
    x_test=x_test[...,None]

print(x_train.shape)
print(x_test.shape)

(1848, 135, 135, 1)
(113, 135, 135, 1)


In [5]:
# architecture of CNN
def buildModel(neurons,drop,nb_batch,hidden_layers):  
  
    nb_epoch=100

    neurons=int(neurons)
    hidden_layers=int(hidden_layers)

    # input layer
    inputs = Input(shape=x_train.shape[1:])

    x = Conv2D(neurons, (3, 3), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)

    # hidden layers
    if hidden_layers !=0:
        for i in range(1,hidden_layers+1):
            x = Conv2D(neurons*(2**i), (3, 3), padding='same', activation='relu')(x)
            x = MaxPooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(neurons*(2**(hidden_layers+1)), activation='relu')(x)
    x = Dropout(drop)(x)

    # output
    predictions = Dense(nb_classes, activation='softmax')(x)

    # modeling
    model = Model(inputs=inputs, outputs=predictions)

    model.compile(optimizer='rmsprop',
                            loss='binary_crossentropy',
                            metrics=['accuracy'])
    
    early_stopping = EarlyStopping(patience=10, verbose=1)

    # learning
    hist = model.fit(x_train, y_train, validation_split=0.2, epochs=int(nb_epoch), batch_size=int(nb_batch),callbacks=[early_stopping])
    
    # evaluate the model with test data
    score = model.evaluate(x_test, y_test)
              
    print("loss : ",score[0])
    print("accuracy : ",score[1])
    
    acc.append(score[1])
    
    return score[0]*(-1)

In [5]:
# functional model

inputs = Input(shape=x_train.shape[1:])

# hyperparameters
drop = 0.3 # dropout rate
hidden_layers = 2 # number of hidden convolutional/max pooling layer sets
batch = 89 # batch size
neurons = 28 # number of convolution kernels

x = Conv2D(neurons, (3, 3), padding='same', activation='relu')(inputs)
x = MaxPooling2D((2, 2), padding='same')(x)

if hidden_layers !=0:
    for i in range(1,hidden_layers+1):
        x = Conv2D(neurons*(2**i), (3, 3), padding='same', activation='relu')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)

x = Flatten()(x)
x = Dense(neurons*(2**(hidden_layers+1)), activation='relu')(x)
x = Dropout(drop)(x)

predictions = Dense(nb_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
display(model.summary())

early_stopping = EarlyStopping(patience=5, verbose=1)

# learning
hist = model.fit(x_train, y_train, validation_split=0.2, epochs=100, batch_size=batch,callbacks=[early_stopping])

# evaluated accuracy for test data
print(model.evaluate(x_test, y_test))

# make graphs
fig, (axL, axR) = plt.subplots(ncols=2,figsize=(16,8))
acc = hist.history['acc'] # accuracy for training data
val_acc = hist.history['val_acc'] # accuracy for validation data
loss = hist.history['loss'] # loss for training data
val_loss = hist.history['val_loss'] # loss for validation
plt.rcParams["font.size"] = 16
axL.plot(acc,label='Training acc')
axL.plot(val_acc,label='Validation acc')
axL.set_title('Accuracy')
axL.legend(loc='best') # position of legend
axR.plot(loss,label='Training loss')
axR.plot(val_loss,label='Validation loss')
axR.set_title('Loss')
axR.legend(loc='best')
plt.show()

In [ ]:
# save model and parameters

model_json = model.to_json()
open(path + "/model/"+model_name+".json", "w").write(model_json)
hdf5_file = path + "/model/"+model_name+".hdf5"
model.save_weights(hdf5_file)

curve = []
curve.append([acc,val_acc,loss,val_loss])
curve = np.array(curve[0])
curve = curve.T
curve_df = pd.DataFrame(curve, columns=["train_acc","val_acc","train_loss","val_loss"])
curve_df.to_csv(path + "/model/" + model_name + "_curve.csv")
print(curve_df)
loss_fin = model.evaluate(x_test,y_test)[0]
acc_fin = model.evaluate(x_test,y_test)[1]
fin_df = pd.DataFrame(np.array([acc_fin,loss_fin]), index=["test_acc","test_loss"])
fin_df.to_csv(path + "/model/" + model_name + "_test-acc.csv")
print(fin_df)

In [ ]:
# load saved model

with open(path+"/model/"+model_name+".json", "r") as f:
    model = model_from_json(f.read())
model.load_weights(path+"/model/"+model_name+".hdf5")
model.summary()